In [25]:
import autogen, os
import json
from dotenv import load_dotenv

load_dotenv("./../credentials_my.env")

cache_seed = 42 # default seed is 41

my_models = [
    # mistral-7b-instruct-v0.1.Q6_K.gguf
    {
        "model": "TheBloke/Mistral-7B-Instruct-v0.1-GGUF/mistral-7b-instruct-v0.1.Q5_0.gguf",
        "api_type": "open_ai",
        "api_base": "http://localhost:1234/v1/chat/completions",
        "api_key":  "sk-XoMik5CKunt0hvEJH5QgT3B1bkFJeoQoffxGuvWBf3G3EFKn"
    }
]


# need for converting to string
os.environ['models_var'] = json.dumps(my_models)

# Setting configurations for autogen
config_list = autogen.config_list_from_json(
    env_or_file='models_var',
    filter_dict={
        "api_base": {
            "http://localhost:1234/v1/chat/completions"
        }
    }
)

config_list

[]

In [20]:
llm_config={
    # "request_timeout": 600,
    "seed": cache_seed,  # seed for caching and reproducibility
    "config_list": config_list,  # a list of OpenAI API configurations
    "temperature": 0,  # temperature for sampling
}

llm_config

{'seed': 42, 'config_list': [], 'temperature': 0}

In [21]:
# create a UserProxyAgent instance named "user_proxy"

user_proxy = autogen.UserProxyAgent(
    name                       = "user_proxy",
    human_input_mode           = "NEVER", # NEVER / ALWAYS / TERMINATE
    max_consecutive_auto_reply = 10,

    # if the x["content"] ends by "TERMINATE", is_termination_msg-->True; otherwise, is_termination_msg--> False
    is_termination_msg=lambda x: x.get("content", "").rstrip().endswith("TERMINATE"),
    
    code_execution_config = {
        "work_dir": "coding",
        
        # Using docker is safer than running the generated code directly.
        # set use_docker=True if docker is available to run the generated code. 
        "use_docker": False
    },
    
    system_message = """Reply TERMINATE if the task has been completed""",
    
    # llm_config = llm_config,
)

user_proxy

In [22]:
# create an AssistantAgent named "assistant"

assistant = autogen.AssistantAgent(
    name="assistant",
    llm_config=llm_config,
    system_message = "You are a coder specializing in Python."
)

assistant

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [23]:
task = """
write a python method to output number 101 to 106
"""

In [24]:
%%time
# the assistant receives a message from the user_proxy, which contains the task description

chat_res = user_proxy.initiate_chat(
    assistant,
    message=task
)

user_proxy (to assistant):


write a python method to output number 101 to 106


--------------------------------------------------------------------------------


TypeError: Completions.create() got an unexpected keyword argument 'api_base'

In [13]:
# autogen.Completion.clear_cache(seed=cache_seed)

In [ ]:
i=1
for cr in chat_res.chat_history:
    print(f"Step {i} --> {cr["content"].rstrip()[-9:]}")
    i=i+1

In [ ]:
# read the content of the cache

import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect(f'./.cache/{cache_seed}/cache.db')

# Create a cursor object
cur = conn.cursor()

# List all tables in the database
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(f"Tables in this DB: {cur.fetchall()}\n")

# Execute a query
cur.execute("SELECT * FROM Cache")

# Fetch and print the results
for row in cur.fetchall():
    print(row)

# Close the connection
conn.close()

In [ ]:
# followup of the previous question
user_proxy.send(
    recipient=assistant,
    message="""Plot a chart of their stock price change YTD. Save the data to stock_price_ytd.csv, and save the plot to stock_price_ytd.png.""",
)

In [ ]:
from IPython.display import display, Image

try:
    image = Image(filename="coding/stock_price_ytd.png")
    display(image)
except FileNotFoundError:
    print("Image not found. Please check the file name and modify if necessary.")